# Libraries we are going to need

In [59]:
import pandas as pd
import re
import numpy as np
from sklearn.preprocessing import LabelEncoder
from pandas.api.types import is_numeric_dtype
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [61]:
le=LabelEncoder()
RFC=RandomForestClassifier(n_estimators=100, random_state=42)
model2 = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    objective='multi:softprob',
    num_class=3)

# Data load 

In [2]:
df= pd.read_csv("data_set/train.csv")


C:\Users\SAKIB\AppData\Local\Temp\ipykernel_14832\1063191192.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv("data_set/train.csv")


# Reviewing data to establish an analytical approach.

In [23]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth',None)
df.head(3)

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3,7,11.27,4.0,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",-1,NaN,11.27,4.0,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3,7,_,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

# Droping the columns that are not needed

In [5]:
ad_df=df.drop(['ID', 'Customer_ID', 'Name', 'SSN', 'Month'],axis=1) #ad in ad_df stands for = after droping

In [6]:
ad_df.head(3)

,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,23,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3,7,11.27,4.0,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,23,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",-1,NaN,11.27,4.0,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,-500,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3,7,_,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good


In [7]:
ad_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 23 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Age                       100000 non-null  object 
 1   Occupation                100000 non-null  object 
 2   Annual_Income             100000 non-null  object 
 3   Monthly_Inhand_Salary     84998 non-null   float64
 4   Num_Bank_Accounts         100000 non-null  int64  
 5   Num_Credit_Card           100000 non-null  int64  
 6   Interest_Rate             100000 non-null  int64  
 7   Num_of_Loan               100000 non-null  object 
 8   Type_of_Loan              88592 non-null   object 
 9   Delay_from_due_date       100000 non-null  int64  
 10  Num_of_Delayed_Payment    92998 non-null   object 
 11  Changed_Credit_Limit      100000 non-null  object 
 12  Num_Credit_Inquiries      98035 non-null   float64
 13  Credit_Mix                100000 non-null  ob

# Handaling null values

In [8]:
ad_df.isna().sum()

Age                             0
Occupation                      0
Annual_Income                   0
Monthly_Inhand_Salary       15002
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                     0
Type_of_Loan                11408
Delay_from_due_date             0
Num_of_Delayed_Payment       7002
Changed_Credit_Limit            0
Num_Credit_Inquiries         1965
Credit_Mix                      0
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age           9030
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly      4479
Payment_Behaviour               0
Monthly_Balance              1200
Credit_Score                    0
dtype: int64

In [9]:
ad_df.shape

(100000, 23)

Monthly_Inhand_Salary (15,002 nulls)

This is missing in 15% of your data. Since salary is highly dependent on Occupation, don't just use a global average. Use the median for each job type to be more accurate.

In [10]:
ad_df['Monthly_Inhand_Salary'] = ad_df['Monthly_Inhand_Salary'].fillna(
    ad_df.groupby('Occupation')['Monthly_Inhand_Salary'].transform('median')
)

Type_of_Loan (11,408 nulls)

You can't calculate a "mean" loan type. The safest move here is to treat "Missing" as its own category.

In [11]:
ad_df['Type_of_Loan'] = ad_df['Type_of_Loan'].fillna("Not Specified")

Monthly_Balance (1,200 nulls) | Amount_invested_monthly (4,479 nulls) | Num_Credit_Inquiries (1,965 nulls) | Num_of_Delayed_Payment (7,002 nulls)

For these columns, using the Median is safer than the Mean because financial data usually has "outliers" (extreme values) that pull the average too high or too low.

In [12]:
cols_to_median = ['Num_of_Delayed_Payment', 'Num_Credit_Inquiries', 
                  'Amount_invested_monthly', 'Monthly_Balance']

for col in cols_to_median:
    # 1. Remove anything that isn't a digit or a decimal point
    ad_df[col] = ad_df[col].astype(str).str.replace('[^0-9.]', '', regex=True)
    # 2. Convert to numeric (this handles the 'nan' strings correctly)
    ad_df[col] = pd.to_numeric(ad_df[col], errors='coerce')
    # 3. Now fill the remaining NaNs with the median
    ad_df[col] = ad_df[col].fillna(ad_df[col].median())

Credit_History_Age (9,030 nulls)

This column usually looks like "22 Years and 3 Months". Before you can fill the nulls, you have to convert those strings into a single number (like total months). Once converted, you can fill the missing values with the median.

In [13]:

def convert_to_months(text):
    # Check if the value is NaN or not a string
    if pd.isna(text) or not isinstance(text, str):
        return None
    
    # Extract all digits from the string (e.g., "22" and "4")
    numbers = re.findall(r'\d+', text)
    
    if len(numbers) == 2:
        years = int(numbers[0])
        months = int(numbers[1])
        return (years * 12) + months
    elif len(numbers) == 1:
        # Handle cases where only years or only months are listed
        return int(numbers[0])
    return None

# 1. Apply the function to create a new numeric column
ad_df['Credit_History_Months'] = ad_df['Credit_History_Age'].apply(convert_to_months)

# 2. Fill the 9,030 missing values with the median
median_history = ad_df['Credit_History_Months'].median()
ad_df['Credit_History_Months'] = ad_df['Credit_History_Months'].fillna(median_history)

# 3. Drop the old text column
ad_df.drop('Credit_History_Age', axis=1, inplace=True)

In [14]:
ad_df.isna().sum()

Age                         0
Occupation                  0
Annual_Income               0
Monthly_Inhand_Salary       0
Num_Bank_Accounts           0
Num_Credit_Card             0
Interest_Rate               0
Num_of_Loan                 0
Type_of_Loan                0
Delay_from_due_date         0
Num_of_Delayed_Payment      0
Changed_Credit_Limit        0
Num_Credit_Inquiries        0
Credit_Mix                  0
Outstanding_Debt            0
Credit_Utilization_Ratio    0
Payment_of_Min_Amount       0
Total_EMI_per_month         0
Amount_invested_monthly     0
Payment_Behaviour           0
Monthly_Balance             0
Credit_Score                0
Credit_History_Months       0
dtype: int64

# Checked If there is any duplicates

In [15]:
ad_df.duplicated().sum()

np.int64(0)

In [16]:
ad_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 23 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Age                       100000 non-null  object 
 1   Occupation                100000 non-null  object 
 2   Annual_Income             100000 non-null  object 
 3   Monthly_Inhand_Salary     100000 non-null  float64
 4   Num_Bank_Accounts         100000 non-null  int64  
 5   Num_Credit_Card           100000 non-null  int64  
 6   Interest_Rate             100000 non-null  int64  
 7   Num_of_Loan               100000 non-null  object 
 8   Type_of_Loan              100000 non-null  object 
 9   Delay_from_due_date       100000 non-null  int64  
 10  Num_of_Delayed_Payment    100000 non-null  float64
 11  Changed_Credit_Limit      100000 non-null  object 
 12  Num_Credit_Inquiries      100000 non-null  float64
 13  Credit_Mix                100000 non-null  ob

# handaling "-" in the data

In [26]:
ad_df= ad_df.replace(('_','-'),0)

In [27]:
ad_df.head(3)

,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Credit_History_Months
0,23,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3,7.0,11.27,4.0,0,809.98,26.822620,No,49.574949,80.415295,High_spent_Small_value_payments,312.494089,Good,265.0
1,23,Scientist,19114.12,3260.465000,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",-1,14.0,11.27,4.0,Good,809.98,31.944960,No,49.574949,118.280222,Low_spent_Large_value_payments,284.629162,Good,219.0
2,-500,Scientist,19114.12,3260.465000,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3,7.0,0,4.0,Good,809.98,28.609352,No,49.574949,81.699521,Low_spent_Medium_value_payments,331.209863,Good,267.0


handaled some outlayers 

In [34]:
ad_df['Age'] = pd.to_numeric(ad_df['Age'].str.replace('_', ''), errors='coerce')
ad_df['Age'] = ad_df['Age'].astype(float)
ad_df.loc[(ad_df['Age'] <= 0) | (ad_df['Age'] > 100), 'Age'] = ad_df['Age'].median()

In [37]:
ad_df['Occupation'] = ad_df['Occupation'].replace('_______', 'Unknown')
ad_df['Payment_Behaviour'] = ad_df['Payment_Behaviour'].replace('!@9#%8', 'Unknown')
ad_df['Payment_of_Min_Amount'] = ad_df['Payment_of_Min_Amount'].replace('NM', 'Not_Specified')

In [38]:
ad_df.head(3)

,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Credit_History_Months
0,23.0,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3,7.0,11.27,4.0,0,809.98,26.822620,No,49.574949,80.415295,High_spent_Small_value_payments,312.494089,Good,265.0
1,23.0,Scientist,19114.12,3260.465000,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",-1,14.0,11.27,4.0,Good,809.98,31.944960,No,49.574949,118.280222,Low_spent_Large_value_payments,284.629162,Good,219.0
2,33.0,Scientist,19114.12,3260.465000,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3,7.0,0,4.0,Good,809.98,28.609352,No,49.574949,81.699521,Low_spent_Medium_value_payments,331.209863,Good,267.0


encoding the data

In [44]:
# Master loop for the entire dataset
for col in ad_df.columns:
    # is_numeric_dtype identifies int64 and float64
    if not is_numeric_dtype(ad_df[col]):
        # Treat as categorical: handle missing values and encode
        ad_df[col] = ad_df[col].fillna('Missing').astype(str)
        ad_df[col] = le.fit_transform(ad_df[col])
    else:
        # Already numeric: just ensure there are no NaNs
        ad_df[col] = ad_df[col].fillna(0)

In [47]:
ad_df.head(3)

,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Credit_History_Months
0,23.0,12,6011,1824.843333,3,4,3,244,128,3,7.0,1186,4.0,0,12062,26.822620,0,49.574949,80.415295,2,312.494089,0,265.0
1,23.0,12,6011,3260.465000,3,4,3,244,128,-1,14.0,1186,4.0,2,12062,31.944960,0,49.574949,118.280222,3,284.629162,0,219.0
2,33.0,12,6011,3260.465000,3,4,3,244,128,3,7.0,660,4.0,2,12062,28.609352,0,49.574949,81.699521,4,331.209863,0,267.0


In [53]:
x = ad_df.drop('Credit_Score', axis=1)
y = ad_df['Credit_Score']

In [54]:
RFC.fit(x,y)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [62]:
model2.fit(x,y)

,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None
